# Homework 1

## 1.1 - Psychologists and AI


I don't believe that simple introspection would be a good way 
for researchers to gain insight into human cognitive processes. 
For some, introspection could be a good method for simple
meditation, or for sorting out one's feelings, however to use
it as an actual scientific discipline seems like a reliance
on bad evidence, especially with the goal of modelling said
processes. 

Through introspection one may believe they know "how" they think,
or how their brain works, but there's really no facts presented
with a conclusion like that. It's mere conjecture. If researchers
truly wanted to model the processes the human brain goes through,
they should set up some kind of experiment that can monitor 
brain activity. I think using introspection does not provide enough
evidence, it's only someone's guesses as to how something works.

## 1.2 - TSP Problem

I set up my TSP "world" as an undirected, interconnected graph.
For testing purposes I made this graph very small and simple.



In [2]:
# Imports
from search import UndirectedGraph, Problem, hill_climbing, simulated_annealing, exp_schedule
import random

In [3]:
TSP_world = UndirectedGraph(dict(
    A=dict(B=1, C=7, D=3, E=6, F=22),
    B=dict(C=5, D=2, E=14, F=3),
    C=dict(D=8, E=12, F=15),
    D=dict(E=10, F=25),
    E=dict(F=20)))


I also took the romania graph, to use a more complicated
example. Because the graph isn't interconnected I created
a function to make it so. 

In [4]:
romania = UndirectedGraph(dict(
    arad=dict(zerind=75, sibiu=140, timisoara=118),
    bucharest=dict(urziceni=85, pitesti=101, giurgiu=90, fagaras=211),
    craiova=dict(dobreta=120, rimnicuvilcea=146, pitesti=138),
    dobreta=dict(mehadia=75),
    eforie=dict(hirsova=86),
    fagaras=dict(sibiu=99),
    hirsova=dict(urziceni=98),
    iasi=dict(vaslui=92, neamt=87),
    lugoj=dict(timisoara=111, mehadia=70),
    oradea=dict(zerind=71, sibiu=151),
    pitesti=dict(rimnicuvilcea=97),
    rimnicuvilcea=dict(sibiu=80),
    urziceni=dict(vaslui=142)))


def interconnect(graph):
    """Interconnect takes a graph and makes it interconnected, connecting each node to every other node"""
    cities = graph.nodes()
    for x in range(len(cities)):
        for y in range(len(cities)):
            if graph.get(cities[x], cities[y]) == None:
                graph.connect(cities[x], cities[y])

For the class problem set up, I set a state to simply be a list
of cities along the route. Actions swapped cities in the route, 
and value is determined by the total path cost.


In [5]:
class TSP(Problem):
    """
    This implements the traveling-sales person problem for local search.
    It requires an undirected graph, with distances between cities.

    State representation:
        [city1, city2, ... , cityn]
        A list of the progression from city to city
    Action representation:
        [index1, index2]
        The index spots of the cities to be swapped in the route order
    """
    def __init__(self, initial, graph):
        self.initial = initial
        self.graph = graph

    def actions(self, state):
        """Actions swaps two cities in the route list"""
        rand = random.randrange(0, len(state), 1)
        rand2 = random.randrange(0, len(state), 1)
        actions = [[rand, rand2]]
        return actions

    def result(self, state, action):
        """Makes the given action on a copy of the board"""
        new_state = state[:]
        temp = new_state[action[0]]
        new_state[action[0]] = new_state[action[1]]
        new_state[action[1]] = temp
        return new_state

    def goal_test(self, state):
        """Checks to see if every city was visited, and if the start and end city are the same"""
        # Get list of all cities
        cities = self.graph.nodes()
        # Check to see if each city is in the given state
        for iter in range(len(cities)):
            if cities[iter] not in state:
                return False

        # Make sure the start and end cities are the same
        if state[0] != state[-1]:
            return False

        return True

    def value(self, state):
        """Adds up the cost of the path for the given state"""
        cost = 0
        for index in range(len(state) - 1):
            if state[index] == state[index+1]: cost += 0
            else:
                cost += self.graph.get(state[index], state[index+1])
        return cost * -1

My implementation can be tried on both the simple TSP world, 
and the romania world.

In [6]:
# Initial route for very simple tsp world
tsp_route = ['D', 'C', 'A', 'B','F', 'E', 'D']

# Initial route for more complicated romania world
interconnect(romania)
romania_route = romania.nodes()
# Make initial state start and end at the same city
romania_route.append(romania_route[0])

# create the problems
# romania world: TSP(romania_route, romania)
# tsp world: TSP(tsp_route, TSP_world)
p = TSP(romania_route, romania)


In [7]:
# Try solving the problem with hill climbing
hill_solution = hill_climbing(p)
print('\nHill-climbing:')
print('\tSolution:\t' + str(hill_solution))
print('\tValue:\t\t' + str(p.value(hill_solution)))
print('\tGoal?\t\t' + str(p.goal_test(hill_solution)))


Hill-climbing:
	Solution:	['lugoj', 'mehadia', 'giurgiu', 'hirsova', 'sibiu', 'rimnicuvilcea', 'zerind', 'vaslui', 'urziceni', 'dobreta', 'fagaras', 'iasi', 'bucharest', 'craiova', 'arad', 'oradea', 'neamt', 'eforie', 'pitesti', 'timisoara', 'lugoj']
	Value:		-419
	Goal?		True


Hill-climbing does an ok job on this problem, especially compared to simulated annealing. It's value isn't
necessarily always the best, but it more often presents a True goal state, which I think is more important in the end.
Still, it doesn't do a great job overall. It's not an ideal solution to the problem.

On the Romania board the best True route hill-climbing found was with a value of -20. 
However this possibly globally optimal solution doesn't appear often. More often I got a local maximum of -170. 

On the TSP world I most often get a solution with value -48. There aren't many frequent other
solutions found. 

Overall, a big issue I've found with hill-climbing for this problem is that it doesn't adjust the 
initial state you give it very much. That initial state is very indicative of what its final
solution will be. This isn't good, because the initial states given are something fairly random, 
not designed to be close to the global solution.

In [8]:
# Try solving the problem with simulated annealing
annealing_solution = \
    simulated_annealing(p, exp_schedule(k=20, lam=0.005, limit=10000))
print('\nSimulated annealing:')
print('\tSolution:\t' + str(annealing_solution))
print('\tValue:\t\t' + str(p.value(annealing_solution)))
print('\tGoal?\t\t' + str(p.goal_test(annealing_solution)))



Simulated annealing:
	Solution:	['lugoj', 'lugoj', 'fagaras', 'arad', 'dobreta', 'hirsova', 'pitesti', 'timisoara', 'giurgiu', 'eforie', 'urziceni', 'sibiu', 'mehadia', 'bucharest', 'iasi', 'rimnicuvilcea', 'oradea', 'neamt', 'vaslui', 'craiova', 'zerind']
	Value:		-19
	Goal?		False


Simulated Annealing often reports better valued solutions than hill-climbing,
but those solutions are often not a goal state. Since solutions don't seem very 
meaningful if they're not true simulated annealing doesn't work as well as hill-climbing.
This surprises me: I thought simulated annealing would be able to get out of the local maximum
given by the initial state, and would make its way to a global max.

These observations were the same for both the TSP world and the romania world. 
There wasn't really a strong functional difference between the two.

## 1.3 - Course Scheduling

For this constraint problem I chose the classes to be the variables that take on the different
values of faculty, time, and classroom. This was the recommended way to set up the problem, 
and also made sense from a logical approach to it. 

The domain is a dictionary with every class being a key, and the value each key contains is
a list of lists containing the combinations of every single class. These combos are in 
the form [faculty, class time, classroom]. 

The neighbors is a dictionary where again every class is a key, and then the values each 
key takes on is every other class, not including itself. Everyone is a neighbor to everyone 
except themselves.

The constraint function takes two classes, with a random combination attached to each 
of them. Constraints are checked to see if this certain pairing breaks any constraints or not.
The benefit of making the values in the domain dictionary a list arises here, because
faculty, time, and room values can all be checked against the constraints, simply by 
indexing the given combo list. This fact is a very important part of making this whole thing work.

With all of this set-up correctly, the CSP class is called, and has pre-defined functions
there to do all the computation. Min-conflicts was used because it's a good function 
for simple conflict problems, like this one.


In [ ]:
"""
Uses the CSP set-up to create a schedule constraint problem.
Classes are the variables, and the different faculty, class times, and classrooms are values that
    the classes can take on.

Created by: Tyler Poel, for CS344, Homework01, at Calvin University
Date: February 27, 2020
"""

from csp import CSP,  min_conflicts, backtracking_search, AC3, parse_neighbors
from search import depth_first_graph_search

def schedule():
    """CSP set-up for a class scheduling problem."""

    classes = 'cs108 cs112 cs214 cs374 cs344 cs212'.split()
    faculty = 'Schuurman Adams Vanderlinden Plantinga'.split()
    times = 'mwf9 mwf10 mwf11 mwf12'.split()
    rooms = 'nh253 sb382'.split()

    # create every [professor, time, room] combo
    combo = [0] * 32
    i = 0
    for x in range(len(faculty)):
        for y in range(len(times)):
            for z in range(len(rooms)):
                combo[i] = [faculty[x], times[y], rooms[z]]
                i += 1

    # Each class gets every single combination
    domains = {}
    for course in classes:
        domains[course] = combo

    # Everyone is a neighbor with everyone but themselves
    neighbors = parse_neighbors("""
            cs108: cs112 cs214 cs374 cs344 cs212;
            cs112: cs214 cs374 cs344 cs212;
            cs214: cs374 cs344 cs212;
            cs374: cs344 cs212;
            cs344: cs212;
            cs212: """)

    def schedule_constraint(A, a, B, b):
        """Codes the constraints the problem must follow"""
        # There can only be one instance of a class
        if A == B:
            return False

        # Certain professors only teach certain classes
        if A == 'cs108' and a[0] != 'Vanderlinden':
            return False
        if B == 'cs108' and b[0] != 'Vanderlinden':
            return False
        if A == 'cs344' and a[0] != 'Vanderlinden':
            return False
        if B == 'cs344' and b[0] != 'Vanderlinden':
            return False
        if A == 'cs374' and a[0] != 'Adams':
            return False
        if B == 'cs374' and b[0] != 'Adams':
            return False
        if A == 'cs112' and a[0] != 'Adams':
            return False
        if B == 'cs112' and b[0] != 'Adams':
            return False
        if A == 'cs212' and a[0] != 'Plantinga':
            return False
        if B == 'cs212' and b[0] != 'Plantinga':
            return False
        if A == 'cs214' and a[0] != 'Schuurman':
            return False
        if B == 'cs214' and b[0] != 'Schuurman':
            return False

        # a professor cannot teach two classes at the same time
        if a[0] == b[0] and a[1] == b[1]:
            return False
        # Two classes cannot be in the same room at the same time
        if a[1] == b[1] and a[2] == b[2]:
            return False

        return True

    return CSP(classes, domains, neighbors, schedule_constraint)


In [ ]:
result = min_conflicts(schedule(), max_steps=1000)

if result is None:
    print("No solution found")
else:
    print("Solution!")
    print(result)


result = min_conflicts(schedule(), max_steps=1000)

if result is None:
    print("No solution found")
else:
    print("Solution!")
    print(result)


In [9]:
"""
Uses the CSP set-up to create a schedule constraint problem.
Classes are the variables, and the different faculty, class times, and classrooms are values that
    the classes can take on.

Created by: Tyler Poel, for CS344, Homework01, at Calvin University
Date: February 27, 2020
"""

from csp import CSP,  min_conflicts, backtracking_search, AC3, parse_neighbors
from search import depth_first_graph_search

def schedule():
    """CSP set-up for a class scheduling problem."""

    classes = 'cs108 cs112 cs214 cs374 cs344 cs212'.split()
    faculty = 'Schuurman Adams Vanderlinden Plantinga'.split()
    times = 'mwf9 mwf10 mwf11 mwf12'.split()
    rooms = 'nh253 sb382'.split()

    # create every [professor, time, room] combo
    combo = [0] * 32
    i = 0
    for x in range(len(faculty)):
        for y in range(len(times)):
            for z in range(len(rooms)):
                combo[i] = [faculty[x], times[y], rooms[z]]
                i += 1

    # Each class gets every single combination
    domains = {}
    for course in classes:
        domains[course] = combo

    # Everyone is a neighbor with everyone but themselves
    neighbors = parse_neighbors("""
            cs108: cs112 cs214 cs374 cs344 cs212;
            cs112: cs214 cs374 cs344 cs212;
            cs214: cs374 cs344 cs212;
            cs374: cs344 cs212;
            cs344: cs212;
            cs212: """)

    def schedule_constraint(A, a, B, b):
        """Codes the constraints the problem must follow"""
        # There can only be one instance of a class
        if A == B:
            return False

        # Certain professors only teach certain classes
        if A == 'cs108' and a[0] != 'Vanderlinden':
            return False
        if B == 'cs108' and b[0] != 'Vanderlinden':
            return False
        if A == 'cs344' and a[0] != 'Vanderlinden':
            return False
        if B == 'cs344' and b[0] != 'Vanderlinden':
            return False
        if A == 'cs374' and a[0] != 'Adams':
            return False
        if B == 'cs374' and b[0] != 'Adams':
            return False
        if A == 'cs112' and a[0] != 'Adams':
            return False
        if B == 'cs112' and b[0] != 'Adams':
            return False
        if A == 'cs212' and a[0] != 'Plantinga':
            return False
        if B == 'cs212' and b[0] != 'Plantinga':
            return False
        if A == 'cs214' and a[0] != 'Schuurman':
            return False
        if B == 'cs214' and b[0] != 'Schuurman':
            return False

        # a professor cannot teach two classes at the same time
        if a[0] == b[0] and a[1] == b[1]:
            return False
        # Two classes cannot be in the same room at the same time
        if a[1] == b[1] and a[2] == b[2]:
            return False

        return True

    return CSP(classes, domains, neighbors, schedule_constraint)


In [10]:
result = min_conflicts(schedule(), max_steps=1000)

if result is None:
    print("No solution found")
else:
    print("Solution!")
    print(result)


Solution!
{'cs108': ['Vanderlinden', 'mwf9', 'sb382'], 'cs112': ['Adams', 'mwf11', 'nh253'], 'cs214': ['Schuurman', 'mwf10', 'sb382'], 'cs374': ['Adams', 'mwf12', 'nh253'], 'cs344': ['Vanderlinden', 'mwf12', 'sb382'], 'cs212': ['Plantinga', 'mwf11', 'sb382']}
